In [15]:
# --- Librerías ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interact

In [42]:
# --- Cargar archivo CSV ---
RUTA = '/workspaces/ml-bootcamp-labs/data/raw/Shift_Pozos_LDiscEoc-cvs.csv'
df = pd.read_csv(RUTA, sep=";")

In [43]:
# Verifica nombres de columnas disponibles
print(df.columns.tolist())

['Well ID', 'UWI', 'Common Well Name', 'X', 'Y', 'Pick Name', 'Pick Z', 'Nearest Node Z', 'Surface Intersection Z', 'Delta', 'Delta Reverse', 'Absolute Delta', 'Interpreter Id', 'Observation #']


In [44]:
# --- Función de estadísticos personalizados ---
def stats(x):
    return pd.Series({
        'count': x.count(),
        'mean': x.mean(),
        'std': x.std(),
        'min': x.min(),
        'p10': np.percentile(x, 10),
        'p50': np.percentile(x, 50),
        'p90': np.percentile(x, 90),
        'max': x.max()
    })

In [48]:
# --- Estadísticos globales ---
print("\nEstadísticos generales de 'Delta':")
print(df['Delta'].describe(percentiles=[0.1, 0.5, 0.9]))


Estadísticos generales de 'Delta':
count    122.000000
mean       6.749800
std      146.858049
min     -328.943573
10%     -178.887060
50%        6.945914
90%      166.233730
max      791.108127
Name: Delta, dtype: float64


In [49]:
# --- Función principal de análisis ---
def analisis(grupo_por, pozo):
    # Agrupación según selección
    stats_group = df.groupby(grupo_por)['Delta'].apply(stats).reset_index()
    
    # Mostrar tabla completa
    display(stats_group)
    
    # Filtrar pozo seleccionado
    if pozo in df[grupo_por].unique():
        df_pozo = df[df[grupo_por] == pozo]
        print(f"\nEstadísticos detallados para {grupo_por} = {pozo}:")
        print(stats(df_pozo['Delta']))
        
        # Histograma + KDE
        plt.figure(figsize=(10,6))
        sns.histplot(df_pozo['Delta'], kde=True, bins=20, color="steelblue")
        plt.title(f"Histograma de Delta - {pozo}")
        plt.show()
        
        # Boxplot comparativo
        plt.figure(figsize=(10,6))
        sns.boxplot(x=grupo_por, y='Delta', data=df[df[grupo_por].isin([pozo])])
        plt.title(f"Boxplot de Delta - {pozo}")
        plt.show()
    
    # Mapa espacial
    plt.figure(figsize=(8,6))
    sc = plt.scatter(df['x'], df['y'], c=df['Delta'], cmap='coolwarm', s=80, edgecolor='k')
    plt.colorbar(sc, label="Delta")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.title("Distribución espacial de Delta")
    plt.show()


In [51]:
# Verifica nombres de columnas disponibles
print(df.columns.tolist())

['Well ID', 'UWI', 'Common Well Name', 'X', 'Y', 'Pick Name', 'Pick Z', 'Nearest Node Z', 'Surface Intersection Z', 'Delta', 'Delta Reverse', 'Absolute Delta', 'Interpreter Id', 'Observation #']


In [52]:
# --- Widgets ---
grupo_selector = widgets.Dropdown(
    options=['UWI', 'Common Well Name'],
    value='UWI',
    description='Agrupar por:'
)

pozo_selector = widgets.Dropdown(
    options=df['UWI'].dropna().unique(),
    description='Pozo:'
)

# Actualizar opciones de pozos según grupo seleccionado
def update_pozo_options(*args):
    pozo_selector.options = df[grupo_selector.value].dropna().unique()

grupo_selector.observe(update_pozo_options, 'value')

# --- Interacción ---
interact(analisis, grupo_por=grupo_selector, pozo=pozo_selector)

<Figure size 800x600 with 0 Axes>

interactive(children=(Dropdown(description='Agrupar por:', options=('UWI', 'Common Well Name'), value='UWI'), …

<function __main__.analisis(grupo_por, pozo)>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>